In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.9 MB/s eta 0:00:00


In [2]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
from torch.utils.data import Dataset
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Lambda, ToTensor


class MNISTDataset(Dataset):
    """MNIST dataset.

    Feature images are automatically flattened.

    Parameters
    ----------
    root : str
        Directory where the actual data is located (or downloaded to).

    train : bool
        If True the training set is returned (60_000 samples). Otherwise
        the validation set is returned (10_000 samples).

    Attributes
    ----------
    tv_dataset : MNIST
        Instance of the torchvision `MNIST` dataset class.
    """

    def __init__(self, root, train=True, download=True):
        transform = Compose(
            [
                ToTensor(),
                Lambda(lambda x: x.ravel()),
            ]
        )

        self.tv_dataset = MNIST(
            root,
            train=train,
            download=download,
            transform=transform,
        )

    def __len__(self):
        """Get the length of the dataset."""
        return len(self.tv_dataset)

    def __getitem__(self, ix):
        """Get a selected sample.

        Parameters
        ----------
        ix : int
            Index of the sample to get.

        Returns
        -------
        x : torch.Tensor
            Flattened feature tensor of shape `(784,)`.

        y : torch.Tensor
            Scalar representing the ground truth label. Number between 0 and 9.
        """
        return self.tv_dataset[ix]


In [ ]:
import math

import torch
import torch.nn as nn
from torch.nn.utils.prune import l1_unstructured, random_unstructured


class MLP(nn.Module):
    """Multilayer perceptron.

    The bias is included in all linear layers.

    Parameters
    ----------
    n_features : int
        Number of input features (pixels inside of MNIST images).

    hidden_layer_sizes : tuple
        Tuple of ints representing sizes of the hidden layers.

    n_targets : int
        Number of target classes (10 for MNIST).

    Attributes
    ----------
    module_list : nn.ModuleList
        List holding all the linear layers in the right order.
    """

    def __init__(self, n_features, hidden_layer_sizes, n_targets):
        super().__init__()

        layer_sizes = (n_features,) + hidden_layer_sizes + (n_targets,)
        layer_list = []

        for i in range(len(layer_sizes) - 1):
            layer_list.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))

        self.module_list = nn.ModuleList(layer_list)

    def forward(self, x):
        """Run the forward pass.

        Parameters
        ----------
        x : torch.Tensor
            Batch of features of shape `(batch_size, n_features)`.

        Returns
        -------
        torch.Tensor
            Batch of predictions (logits) of shape `(batch_size, n_targets)`.
        """
        n_layers = len(self.module_list)

        for i, layer in enumerate(self.module_list):
            x = layer(x)

            if i < n_layers - 1:
                x = nn.functional.relu(x)

        return x


def prune_linear(linear, prune_ratio=0.3, method="l1"):
    """Prune a linear layer.

    Modifies the module in-place. We make an assumption that the bias
    is included.

    Parameters
    ----------
    linear : nn.Linear
        Linear module containing a bias.

    prune_ratio : float
        Number between 0 and 1 representing the percentage of weights
        to prune.

    method : str, {"l1", "random"}
        Pruning method to use.
    """
    if method == "l1":
        prune_func = l1_unstructured
    elif method == "random":
        prune_func = random_unstructured
    else:
        raise ValueError

    prune_func(linear, "weight", prune_ratio)
    prune_func(linear, "bias", prune_ratio)


def prune_mlp(mlp, prune_ratio=0.3, method="l1"):
    """Prune each layer of the multilayer perceptron.

    Modifies the module in-place. We make an assumption that each
    linear layer has the bias included.

    Parameters
    ----------
    mlp : MLP
        Multilayer perceptron instance.

    prune_ratio : float or list
        Number between 0 and 1 representing the percentage of weights
        to prune. If `list` then different ratio for each
        layer.

    method : str, {"l1", "random"}
        Pruning method to use.
    """
    if isinstance(prune_ratio, float):
        prune_ratios = [prune_ratio] * len(mlp.module_list)
    elif isinstance(prune_ratio, list):
        if len(prune_ratio) != len(mlp.module_list):
            raise ValueError("Incompatible number of prune ratios provided")

        prune_ratios = prune_ratio
    else:
        raise TypeError

    for prune_ratio, linear in zip(prune_ratios, mlp.module_list):
        prune_linear(linear, prune_ratio=prune_ratio, method=method)


def check_pruned_linear(linear):
    """Check if a Linear module was pruned.

    We require both the bias and the weight to be pruned.

    Parameters
    ----------
    linear : nn.Linear
        Linear module containing a bias.

    Returns
    -------
    bool
        True if the model has been pruned.
    """
    params = {param_name for param_name, _ in linear.named_parameters()}
    expected_params = {"weight_orig", "bias_orig"}

    return params == expected_params


def reinit_linear(linear):
    """Reinitialize a linear layer.

    This is an in-place operation.
    If the module has some pruning logic we are not going to remove it
    and we only initialize the underlying tensors - `weight_orig` and
    `bias_orig`.

    Parameters
    ----------
    linear : nn.Linear
        Linear model containing a bias.
    """
    is_pruned = check_pruned_linear(linear)

    # Get parameters of interest
    if is_pruned:
        weight = linear.weight_orig
        bias = linear.bias_orig
    else:
        weight = linear.weight
        bias = linear.bias

    # Initialize weight
    nn.init.kaiming_uniform_(weight, a=math.sqrt(5))

    # Initialize bias
    fan_in, _ = nn.init._calculate_fan_in_and_fan_out(weight)
    bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
    nn.init.uniform_(bias, -bound, bound)


def reinit_mlp(mlp):
    """Reinitialize all layers of the MLP.

    Parameters
    ----------
    mlp : MLP
        Multi-layer perceptron.
    """
    for linear in mlp.module_list:
        reinit_linear(linear)


def copy_weights_linear(linear_unpruned, linear_pruned):
    """Copy weights from an unpruned model to a pruned model.

    Modifies `linear_pruned` in place.

    Parameters
    ----------
    linear_unpruned : nn.Linear
        Linear model with a bias that was not pruned.

    linear_pruned : nn.Linear
        Linear model with a bias that was pruned.
    """
    assert check_pruned_linear(linear_pruned)
    assert not check_pruned_linear(linear_unpruned)

    with torch.no_grad():
        linear_pruned.weight_orig.copy_(linear_unpruned.weight)
        linear_pruned.bias_orig.copy_(linear_unpruned.bias)


def copy_weights_mlp(mlp_unpruned, mlp_pruned):
    """Copy weights of an unpruned network to a pruned network.

    Modifies `mlp_pruned` in place.

    Parameters
    ----------
    mlp_unpruned : MLP
        MLP model that was not pruned.

    mlp_pruned : MLP
        MLP model that was pruned.
    """
    zipped = zip(mlp_unpruned.module_list, mlp_pruned.module_list)

    for linear_unpruned, linear_pruned in zipped:
        copy_weights_linear(linear_unpruned, linear_pruned)


def compute_stats(mlp):
    """Compute important statistics related to pruning.

    Parameters
    ----------
    mlp : MLP
        Multilayer perceptron.

    Returns
    -------
    dict
        Statistics.
    """
    stats = {}
    total_params = 0
    total_pruned_params = 0

    for layer_ix, linear in enumerate(mlp.module_list):
        assert check_pruned_linear(linear)

        weight_mask = linear.weight_mask
        bias_mask = linear.bias_mask

        params = weight_mask.numel() + bias_mask.numel()
        pruned_params = (weight_mask == 0).sum() + (bias_mask == 0).sum()

        total_params += params
        total_pruned_params += pruned_params

        stats[f"layer{layer_ix}_total_params"] = params
        stats[f"layer{layer_ix}_pruned_params"] = pruned_params
        stats[f"layer{layer_ix}_actual_prune_ratio"] = pruned_params / params

    stats["total_params"] = total_params
    stats["total_pruned_params"] = total_pruned_params
    stats["actual_prune_ratio"] = total_pruned_params / total_params

    return stats


In [ ]:
import torch
import torch.nn as nn
import tqdm
from torch.utils.data import DataLoader

import wandb
# from data import MNISTDataset
# from utils import MLP, compute_stats, copy_weights_mlp, prune_mlp, reinit_mlp


def loop_dataloader(dataloader):
    """Loop infinitely over a dataloader.

    Parameters
    ----------
    dataloader : DataLoader
        DataLoader streaming batches of samples.

    Yields
    ------
    X_batch : torch.Tensor
        Batch of features.

    y_batch : torch.Tensor
        Batch of predictions.
    """
    while True:
        for x in iter(dataloader):
            yield x


def train(model, dataloader_train, loss_inst, optimizer, max_iter=10_000,
          dataloader_val=None, val_freq=500):
    """Run the training loop.

    Parameters
    ----------
    model : nn.Module
        Neural network (in our case MLP).

    dataloader_train : DataLoader
        Dataloader yielding training samples.

    loss_inst : callable
        Computes the loss when called.

    optimizer : torch.optim.Optimizer
        Instance of an optimizer.

    max_iter : int
        The number of iterations we run the training for
        (= number of graident descent steps).

    dataloader_val : None or DataLoader
        Dataloader yielding validation samples. If provided it will
        also single to us that we want to track metrics.

    val_freq : int
        How often evaluation run.
    """
    global device
    iterable = loop_dataloader(dataloader_train)
    iterable = tqdm.tqdm(iterable, total=max_iter)
    it = 0
    for X_batch, y_batch in iterable:
        if it == max_iter:
            break
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        logit_batch = model(X_batch)

        loss = loss_inst(logit_batch, y_batch)
        if dataloader_val is not None:
            wandb.log({"loss": loss}, step=it)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if it % val_freq == 0 and dataloader_val is not None:
            is_equal = []

            for X_batch_val, y_batch_val in dataloader_val:
                X_batch_val = X_batch_val.to(device)
                y_batch_val = y_batch_val.to(device)
                is_equal.append(
                    model(X_batch_val).argmax(dim=-1) == y_batch_val
                )

            is_equal_t = torch.cat(is_equal)
            acc = is_equal_t.sum() / len(is_equal_t)
            wandb.log({"accuracy_val": acc}, step=it)

        it += 1



def experiment(MAX_ITERS, PRUNE_ITERS, PRUNE_METHODS, PRUNE_RATIOS, REINITIALIZES , RANDOM_STATES, project_name, start_from):
    args = {"max-iter" : 50000, "batch-size" : 64, "prune-iter" : 1,
          "prune-method" : "l1", "prune-ratio" : 0.2, "val-freq" : 250,
          "reinitialize" : "false", "random-state" : 1}
    i = 0;
    for max_iter in MAX_ITERS:
      for prune_iter in PRUNE_ITERS:
        for prune_method in PRUNE_METHODS:
          for prune_ratio in PRUNE_RATIOS:
            for reinit in REINITIALIZES:
              for random_state in RANDOM_STATES:
                if i < start_from:
                  i += 1
                  continue
                args["max-iter"] = max_iter
                args["prune-iter"] = prune_iter
                args["prune-method"] = prune_method
                args["prune-ratio"] = prune_ratio
                args["reinitialisalize"] = reinit
                args["random-state"] = random_state

                wandb.init(
                    project=project_name,
                    entity="bspanfilov",
                    config=args,
                )
                wandb.define_metric("accuracy_val", summary="max")

                dataset_train = MNISTDataset(
                    "data",
                    train=True,
                    download=True,
                )
                dataset_val = MNISTDataset(
                    "data",
                    train=False,
                    download=True,
                )

                if args["random-state"] is not None:
                    torch.manual_seed(args["random-state"])

                dataloader_train = DataLoader(
                    dataset_train, batch_size=args["batch-size"], shuffle=True
                )
                dataloader_val = DataLoader(
                    dataset_val, batch_size=args["batch-size"], shuffle=True
                )

                kwargs = dict(
                    n_features=28 * 28,
                    hidden_layer_sizes=(300, 100),
                    n_targets=10,
                )

                mlp = MLP(**kwargs).to(device)

                mlp_copy = MLP(**kwargs).to(device)
                mlp_copy.load_state_dict(mlp.state_dict())

                loss_inst = nn.CrossEntropyLoss()
                optimizer = torch.optim.Adam(mlp.parameters(), lr=1.2 * 1e-3)

                # Train and prune loop
                if args["prune-ratio"] > 0:
                    per_round_prune_ratio = 1 - (1 - args["prune-ratio"]) ** (1 / args["prune-iter"])

                    per_round_prune_ratios = [per_round_prune_ratio] * len(mlp.module_list)
                    per_round_prune_ratios[-1] /= 2

                    per_round_max_iter = int(args["max-iter"] / args["prune-iter"])

                    for prune_it in range(args["prune-iter"]):
                        train(
                            mlp,
                            dataloader_train,
                            loss_inst,
                            optimizer,
                            max_iter=per_round_max_iter,
                        )
                        prune_mlp(mlp, per_round_prune_ratios, method=args["prune-method"])

                        copy_weights_mlp(mlp_copy, mlp)
                        # надо будет добавить эксперементов без копирования, вдруг тоже что-то получиться
                        # а пока что фигачу кучу моделей на mnist

                        stats = compute_stats(mlp)
                        for name, stat in stats.items():
                            summary_name = f"{name}_pruneiter={prune_it}"
                            wandb.run.summary[summary_name] = stat

                if args["reinitialize"] == "true":
                    reinit_mlp(mlp)
                # Run actual training with a final pruned network
                train(
                    mlp,
                    dataloader_train,
                    loss_inst,
                    optimizer,
                    max_iter=args["max-iter"],
                    dataloader_val=dataloader_val,
                    val_freq=args["val-freq"],
                )




In [ ]:
# тут просто настройки, с которыми я как раз запускал эксперименты
MAX_ITERS=[13000]
PRUNE_ITERS=[1, 5, 8]
PRUNE_METHODS=['l1', 'random']
PRUNE_RATIOS=[0.97]
REINITIALIZES=['false', 'true']
RANDOM_STATES=[1, 3]

experiment(MAX_ITERS, PRUNE_ITERS, PRUNE_METHODS, PRUNE_RATIOS, REINITIALIZES, RANDOM_STATES, "experiment from the title", 0)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

...

100%|██████████| 13000/13000 [04:11<00:00, 51.75it/s]


accuracy_val,▁▁▃▇▇▇██████████████████████████████████
loss,██▃▂▂▃▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▂▁
actual_prune_ratio_pruneiter=0,0.3542
actual_prune_ratio_pruneiter=1,0.58283
actual_prune_ratio_pruneiter=2,0.73042
actual_prune_ratio_pruneiter=3,0.82572
actual_prune_ratio_pruneiter=4,0.88725
actual_prune_ratio_pruneiter=5,0.92702
actual_prune_ratio_pruneiter=6,0.95271
actual_prune_ratio_pruneiter=7,0.96932
layer0_actual_prune_ratio_pruneiter=0,0.35488


100%|██████████| 13000/13000 [04:11<00:00, 51.59it/s]


accuracy_val,▁▂▅▇▇▇██████████████████████████████████
loss,█▆▃▃▂▂▂▂▂▂▂▂▂▂▁▃▁▂▂▂▁▁▁▁▂▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁
actual_prune_ratio_pruneiter=0,0.3542
actual_prune_ratio_pruneiter=1,0.58283
actual_prune_ratio_pruneiter=2,0.73042
actual_prune_ratio_pruneiter=3,0.82572
actual_prune_ratio_pruneiter=4,0.88725
actual_prune_ratio_pruneiter=5,0.92702
actual_prune_ratio_pruneiter=6,0.95271
actual_prune_ratio_pruneiter=7,0.96932
layer0_actual_prune_ratio_pruneiter=0,0.35488


100%|██████████| 13000/13000 [04:14<00:00, 51.09it/s]
